In [73]:
import sys, os
import networkx as nx
import ipycytoscape
import json

sys.path.append('../frontends/pyscripts')
from matfuncs import *

S = PolynomialRing(ZZ, 't')

In [74]:
def kl(M):
    return M.lattice_of_flats().kazhdan_lusztig_polynomial()(t)

def parallel_connection(m, n):
    G = graphs.CycleGraph(m + n - 2)
    G.add_edge(0, m-1)
    edge_e = frozenset({(0, m - 1)})
    return G, edge_e

def qhat(M):
    return (-1) ^ M.rank() * invkl(M)

def uniformQhat(r, n):
    return (-1) ^ r * uniformQpoly(r, n)

def sum_prhc(M, S):
    return sum(kl(M.delete(M.groundset() - set(F))) * \
            qhat(M.contract(F)) for F in S)

def sum_pchr(M, S):
    return sum(qhat(M.delete(M.groundset() - set(F))) * \
            kl(M.contract(F)) for F in S)

def prhc_at_flats(M, L):
    return {F: S(kl(M.delete(M.groundset() - set(F))) * qhat(M.contract(F))) for F in L}

In [75]:
def save_to_json(m, n, M):
    # Generate the JSON data
    L = M.lattice_of_flats()
    flat_to_index = {flat: i for i, flat in enumerate(L)}
    prhc_vals = prhc_at_flats(M, list(L))
    
    # Create parent nodes for each rank
    ranks = [i for i in range(M.rank() + 1)]
    rank_parents = {r: f'rank_{r}' for r in ranks}
    rank_colors = ['#DFDF00', '#00F0F0', '#FFFF00', '#FFD0DF', '#D3D3D3', '#ADD8E6', '#E0FFFF']
    
    nodes = []
    edges = []
    
    # Add rank parent nodes
    for r in ranks:
        nodes.append({
            "data": {
                "id": rank_parents[r],
                "label": f"Rank {r}",
                "color": '#CCCCCC'  # Light gray for parent nodes
            }
        })
    
    # Add child nodes with parent relationships
    for i, flat in enumerate(L):
        nodes.append({
            "data": {
                "id": str(i),
                "label": f"{prhc_vals.get(flat, 0)}",
                "color": rank_colors[M.rank(flat) % len(rank_colors)],
                "parent": rank_parents[M.rank(flat)]
            }
        })
    
    # Add edges between children
    for x, y in L.cover_relations():
        edges.append({
            "data": {
                "source": str(flat_to_index[y]),
                "target": str(flat_to_index[x])
            }
        })
    
    # Create final JSON structure
    cyto_data = {
        "nodes": nodes,
        "edges": edges
    }
    
    # Save the data to a JSON file
    with open(f"graph_data_pcr-pc-{m}-{n}-{str(M)}.json", "w") as f:
        json.dump(cyto_data, f, indent=4)
    
    print(f"JSON file saved as graph_data_pcr-pc-{m}-{n}-{str(M)}.json")

In [76]:
m, n = 4, 7

G, e = parallel_connection(m, n)
M = Matroid(G)
# Usage
save_to_json(m, n, M)

JSON file saved as graph_data_pcr-pc-4-7-Graphic matroid of rank 8 on 10 elements.json
